In [3]:
import requests
import json
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from utils import pressure

In [4]:
def get_trade_price():
    url = "https://api.upbit.com/v1/ticker"
    headers = {"accept": "application/json"}
    query_params = {"markets": "KRW-BTC"}

    response = requests.get(url, headers=headers, params=query_params)
    res = json.loads(response.text)[0]
    return res['timestamp'], res['trade_price']

def get_orderbook_data():
    headers = {"accept": "application/json"}
    query_params = {"markets": "KRW-BTC", "count":1}

    url = f"https://api.upbit.com/v1/orderbook"
    response = requests.get(url, headers=headers, params=query_params)
    return json.loads(response.text)

In [9]:
def save_pressure():
  data = get_orderbook_data()
  p = pressure(data)
  res = requests.post("http://127.0.0.1:8000/api/pressures/",data={
    "timestamp":data[0]['timestamp'], 
    "ask_pressure": p['ask'],
    "bid_pressure": p['bid'],
    "pressure_ratio": (p['ask'] - p['bid'])/(p['ask'] + p['bid'])
  })

  return res
  

In [19]:
def save_price():
  t, price = get_trade_price()

  res = requests.post("http://127.0.0.1:8000/api/trade-prices/",data={
    "timestamp":t, 
    "price": price
  })

  return res
  

In [6]:
save_pressure()

<Response [201]>

In [22]:
for i in tqdm(range(10)):
  save_pressure()
  time.sleep(1)

100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


In [17]:
print(save_price().text)

1721449360118 93412000.0
{"id":1,"timestamp":1721449360118,"price":93412000.0}


In [21]:
for i in tqdm(range(10)):
  save_price()
  time.sleep(1)

100%|██████████| 10/10 [00:11<00:00,  1.10s/it]
